<a href="https://colab.research.google.com/github/Kriti-be21/Stock-Price-Prediction/blob/main/MSFT_DATA_MINING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

All the news have been extracted from https://business.financialpost.com/

In [ ]:
import requests
import urllib
import sys
import numpy  as np
import pandas as pd

from bs4 import BeautifulSoup

# 1. Extract web links according to a signature and the number of pages


In [ ]:
def get_internal_links(ur):

    info  = BeautifulSoup(requests.get(ur,allow_redirects = True).content,'html.parser').find_all("div",{"class":"article-card__details"})
    links  = [ "https://financialpost.com" + i.a['href'] for i in info]

    return(links)

In [ ]:
def extract_link_of_news(k_word,n_of_page):
    link= "https://financialpost.com/search/?search_text="+ k_word + "&date_range=-3650d&sort=score&from=" + str((n_of_page-1)*10)
    news_df                  = pd.DataFrame()

    news_df["internal_urls"]  = get_internal_links(link)
    news_df["principal_url"]  = link
    news_df["n_of_page"]      = n_of_page
    return(news_df[["n_of_page","principal_url","internal_urls"]])

In [ ]:
n_of_pages = 988
df = pd.concat([extract_link_of_news("microsoft",i) for i in range(1,n_of_pages+1)],ignore_index = True)
df

,n_of_page,principal_url,internal_urls
0,1,https://financialpost.com/search/?search_text=...,https://financialpost.com/pmn/press-releases-p...
1,1,https://financialpost.com/search/?search_text=...,https://financialpost.com/pmn/business-pmn/mic...
2,1,https://financialpost.com/search/?search_text=...,https://financialpost.com/technology/tech-news...
3,1,https://financialpost.com/search/?search_text=...,https://financialpost.com/globe-newswire/quisi...
4,1,https://financialpost.com/search/?search_text=...,https://financialpost.com/investing/microsoft-...
...,...,...,...
9875,988,https://financialpost.com/search/?search_text=...,https://financialpost.com/women-of-power/canad...
9876,988,https://financialpost.com/search/?search_text=...,https://financialpost.com/pmn/press-releases-p...
9877,988,https://financialpost.com/search/?search_text=...,https://financialpost.com/pmn/press-releases-p...
9878,988,https://financialpost.com/search/?search_text=...,https://financialpost.com/pmn/press-releases-p...


# 2. Extract the date and text from each link

In [ ]:
def extract_date_and_text(ur):
    url = ur

    try:
        soup         = BeautifulSoup(requests.get(url).content, 'html.parser')
        g = 0

    except:
        the_type, the_value, the_traceback = sys.exc_info()
        g = 1


    if(g==0):
        flag= True
        if(soup.find("section",class_ = "article-content__content-group")):
            text_of_news = soup.find("section",class_ = "article-content__content-group").find_all("p")
            flag=False
        else:
            text_of_news = "NO TEXT"


        if(flag):
            final_date = ""
            final_text =""
        else:
            final_date= soup.find("span",class_="published-date__since").text
            get_text_vec = [i.text for i in text_of_news]
            final_text = "".join(get_text_vec)
        #final_date   = soup.find("span",class_="published-date__since").text["datetime"]


    else: final_date = "";final_text = ""

    return(final_date,final_text)

In [ ]:
a = np.vectorize(extract_date_and_text)
test = df[0:10].copy()
test["internals_dates"],test["internals_text"] = a(test["internal_urls"][0:10])
test

,n_of_page,principal_url,internal_urls,internals_dates,internals_text
0,1,https://financialpost.com/search/?search_text=...,https://financialpost.com/pmn/press-releases-p...,"Oct 04, 2021",NEW YORK — Telstra today announced it is a sel...
1,1,https://financialpost.com/search/?search_text=...,https://financialpost.com/pmn/business-pmn/mic...,"Nov 02, 2021",Microsoft Corp on Tuesday announced a new roun...
2,1,https://financialpost.com/search/?search_text=...,https://financialpost.com/technology/tech-news...,"Mar 05, 2021",Although the flagship launch at Microsoft Igni...
3,1,https://financialpost.com/search/?search_text=...,https://financialpost.com/globe-newswire/quisi...,"Jul 08, 2021","TORONTO, July 08, 2021 (GLOBE NEWSWIRE) — Quis..."
4,1,https://financialpost.com/search/?search_text=...,https://financialpost.com/investing/microsoft-...,"Oct 27, 2021",A blowout first quarter has brought Microsoft ...
5,1,https://financialpost.com/search/?search_text=...,https://financialpost.com/pmn/business-pmn/mic...,"Nov 16, 2021",(Bloomberg) — Microsoft Corp. has turned on a ...
6,1,https://financialpost.com/search/?search_text=...,https://financialpost.com/pmn/press-releases-p...,"May 13, 2021","VANCOUVER, British Columbia — Microsoft Dynami..."
7,1,https://financialpost.com/search/?search_text=...,https://financialpost.com/globe-newswire/quisi...,"Jul 29, 2021","TORONTO, July 29, 2021 (GLOBE NEWSWIRE) — Quis..."
8,1,https://financialpost.com/search/?search_text=...,https://financialpost.com/globe-newswire/degre...,"Oct 05, 2021","PLEASANTON, Calif., Oct. 05, 2021 (GLOBE NEWSW..."
9,1,https://financialpost.com/search/?search_text=...,https://financialpost.com/pmn/press-releases-p...,"Mar 16, 2021",PARIS LA DÉFENSE — Thales announces a new data...


In [ ]:
a = np.vectorize(extract_date_and_text)
df["internals_dates"],df["internals_text"] = a(df["internal_urls"])

In [ ]:
df

,n_of_page,principal_url,internal_urls,internals_dates,internals_text
0,1,https://financialpost.com/search/?search_text=...,https://financialpost.com/pmn/press-releases-p...,"Oct 04, 2021",NEW YORK — Telstra today announced it is a sel...
1,1,https://financialpost.com/search/?search_text=...,https://financialpost.com/pmn/business-pmn/mic...,"Nov 02, 2021",Microsoft Corp on Tuesday announced a new roun...
2,1,https://financialpost.com/search/?search_text=...,https://financialpost.com/technology/tech-news...,"Mar 05, 2021",Although the flagship launch at Microsoft Igni...
3,1,https://financialpost.com/search/?search_text=...,https://financialpost.com/globe-newswire/quisi...,"Jul 08, 2021","TORONTO, July 08, 2021 (GLOBE NEWSWIRE) — Quis..."
4,1,https://financialpost.com/search/?search_text=...,https://financialpost.com/investing/microsoft-...,"Oct 27, 2021",A blowout first quarter has brought Microsoft ...
...,...,...,...,...,...
9875,988,https://financialpost.com/search/?search_text=...,https://financialpost.com/women-of-power/canad...,"Dec 02, 2014",Authentic leaders know themselves — their stre...
9876,988,https://financialpost.com/search/?search_text=...,https://financialpost.com/pmn/press-releases-p...,"Aug 13, 2019",Fiscal 2019 Net Income Increases 144% Over Pri...
9877,988,https://financialpost.com/search/?search_text=...,https://financialpost.com/pmn/press-releases-p...,"Nov 12, 2019",Continued Momentum in Enterprise and Governmen...
9878,988,https://financialpost.com/search/?search_text=...,https://financialpost.com/pmn/press-releases-p...,"Nov 09, 2020",Q1 Results include Record 11% revenue growth; ...


In [ ]:
df.loc[df["internals_text"] == "NO TEXT"]

,n_of_page,principal_url,internal_urls,internals_dates,internals_text


In [ ]:
len(df.loc[df["internals_text"] == ""])

55

In [ ]:
df.loc[df["internals_text"] == ""]

,n_of_page,principal_url,internal_urls,internals_dates,internals_text
354,36,https://financialpost.com/search/?search_text=...,https://financialpost.com/pmn/press-releases-p...,"Jun 30, 2021",
1042,105,https://financialpost.com/search/?search_text=...,https://financialpost.com/pmn/press-releases-p...,"Nov 24, 2020",
1294,130,https://financialpost.com/search/?search_text=...,https://financialpost.com/technology/tech-news...,"Jan 17, 2020",
1464,147,https://financialpost.com/search/?search_text=...,https://financialpost.com/technology/tech-news...,"Jul 28, 2020",
1470,148,https://financialpost.com/search/?search_text=...,https://financialpost.com/technology/tech-news...,"Jul 28, 2020",
1616,162,https://financialpost.com/search/?search_text=...,https://financialpost.com/technology/rims-opti...,"Jun 29, 2012",
1633,164,https://financialpost.com/search/?search_text=...,https://financialpost.com/technology/rims-opti...,"Jun 29, 2012",
1749,175,https://financialpost.com/search/?search_text=...,https://financialpost.com/technology/tech-news...,"May 27, 2020",
1905,191,https://financialpost.com/search/?search_text=...,https://financialpost.com/pmn/press-releases-p...,"Mar 31, 2020",
2277,228,https://financialpost.com/search/?search_text=...,https://financialpost.com/pmn/press-releases-p...,"May 18, 2021",


In [ ]:
u=df['internals_urls'][1294]
u

'https://financialpost.com/technology/tech-news/windows-7-is-dead-but-25-of-large-businesses-in-canada-are-still-running-it-on-desktops-says-idc-survey'

In [ ]:
extract_date_and_text(u)

('Jan 17, 2020', '')

In [ ]:
s = BeautifulSoup(requests.get(u).content, 'html.parser')

In [ ]:
text=s.find_all("section",class_ = "article-content__content-group")

In [ ]:
res=[i.find_all("p") for i in text]

In [ ]:
res

[[],
 [<p>Windows 7 is officially dead.</p>,
  <p data-async="">No, it won’t suddenly cease to function; as anyone who booted it up after the end-of-support deadline of January 14 discovered. It works just fine after you get past the full-screen warning from <a data-evt="click" data-evt-typ="User Interaction Click" data-evt-val='{"control_fields": {"mparticle": {"keys": {"Click Source Type": "Click Source Type", "Click Source Name": "Click Source Name", "Click Target URL": "Click Target URL"}, "mp_event_type": "Navigation", "extra_keys": ["Click Source Vertical Position %", "Click Source Vertical Position Pixels"]}}, "Click Source Type": "in-page link", "Click Source Name": "Microsoft", "Click Target URL": "https://www.itworldcanada.com/tag/microsoft"}' href="https://www.itworldcanada.com/tag/microsoft" rel="noopener noreferrer" target="_blank">Microsoft</a> that you’re now at risk since you’re running an unsupported operating system.</p>,
  <p>And they’re right. Once a product is out 

In [ ]:
df['internals_text'][0]

'NEW YORK — Telstra today announced it is a selected carrier for Microsoft Azure Peering Service, a networking service that enhances customer connectivity, and Operator Connect for Microsoft Teams, a new programmatic approach to operator-provided calling in Teams.  Today’s announcement builds on the long-standing relationship between Telstra and Microsoft to help enhance connectivity to Microsoft cloud services and provide an integrated calling experience via Teams. '

In [ ]:
df.to_pickle("/content/drive/MyDrive/custom.pkl")

In [ ]:
df= pd.read_pickle("/content/drive/MyDrive/custom.pkl")

In [ ]:
df.loc[df["internals_text"] == ""]

,n_of_page,principal_url,internal_urls,internals_dates,internals_text
354,36,https://financialpost.com/search/?search_text=...,https://financialpost.com/pmn/press-releases-p...,"Jun 30, 2021",
1042,105,https://financialpost.com/search/?search_text=...,https://financialpost.com/pmn/press-releases-p...,"Nov 24, 2020",
1294,130,https://financialpost.com/search/?search_text=...,https://financialpost.com/technology/tech-news...,"Jan 17, 2020",
1464,147,https://financialpost.com/search/?search_text=...,https://financialpost.com/technology/tech-news...,"Jul 28, 2020",
1470,148,https://financialpost.com/search/?search_text=...,https://financialpost.com/technology/tech-news...,"Jul 28, 2020",
1616,162,https://financialpost.com/search/?search_text=...,https://financialpost.com/technology/rims-opti...,"Jun 29, 2012",
1633,164,https://financialpost.com/search/?search_text=...,https://financialpost.com/technology/rims-opti...,"Jun 29, 2012",
1749,175,https://financialpost.com/search/?search_text=...,https://financialpost.com/technology/tech-news...,"May 27, 2020",
1905,191,https://financialpost.com/search/?search_text=...,https://financialpost.com/pmn/press-releases-p...,"Mar 31, 2020",
2277,228,https://financialpost.com/search/?search_text=...,https://financialpost.com/pmn/press-releases-p...,"May 18, 2021",


In [ ]:
cnt=0
for i in df['internals_text']:
    if(i==''):
        df.drop(index=cnt, inplace=True)
    cnt+=1

In [ ]:
df.loc[df["internals_text"] == ""]

,n_of_page,principal_url,internal_urls,internals_dates,internals_text


In [ ]:
df.to_pickle("/content/drive/MyDrive/custom.pkl")